# Data Wrangling and Cleaning

In [1]:
import pandas as pd

In [4]:
df_derechos_laborales = pd.read_csv(r'migracion_charly\regulacion_juridica_al_empleo.csv', sep=',', header=0)

Elimino columnas irrelevantes para el proyecto y/o redundantes

In [5]:
df_derechos_laborales.drop('source', axis=1, inplace=True)
df_derechos_laborales.drop('source.label', axis=1, inplace=True)
df_derechos_laborales.drop('indicator', axis=1, inplace=True)
df_derechos_laborales.drop('indicator.label',axis=1, inplace=True)

Cambio el nombre de las columnas

In [6]:
df_derechos_laborales.rename(columns={'ref_area':'codigo_pais', 'ref_area.label':'pais','time':'anio','obs_value':'derechos_laborales'}, inplace=True)

In [7]:
df_derechos_laborales

,codigo_pais,pais,anio,derechos_laborales
0,AGO,Angola,2020,2.27
1,AGO,Angola,2019,2.27
2,AGO,Angola,2018,2.27
3,ALB,Albania,2020,0.72
4,ALB,Albania,2019,0.72
...,...,...,...,...
1231,ZWE,Zimbabwe,2019,4.42
1232,ZWE,Zimbabwe,2018,5.92
1233,ZWE,Zimbabwe,2017,4.96
1234,ZWE,Zimbabwe,2016,4.95


In [9]:
df_human_rights = pd.read_csv(r'migracion_charly\distribution-human-rights.csv', sep=',', header=0)

In [10]:
df_human_rights.rename(columns={'Entity':'pais', 'Code':'codigo_pais', 'Year':'anio', 'civ_libs_vdem_owid': 'derechos_humanos'}, inplace=True)

In [11]:
df_human_rights.dropna(subset=['derechos_humanos'], inplace=True)

In [12]:
df_human_rights.drop('codigo_pais', axis=1, inplace=True)

In [13]:
df_human_rights = df_human_rights[(df_human_rights['anio'] >= 1990)]

In [14]:
df_human_rights

,pais,anio,derechos_humanos,region
201,Afghanistan,1990,0.077,Asia
202,Afghanistan,1991,0.069,Asia
203,Afghanistan,1992,0.071,Asia
204,Afghanistan,1993,0.077,Asia
205,Afghanistan,1994,0.078,Asia
...,...,...,...,...
33621,Zimbabwe,2017,0.434,Africa
33622,Zimbabwe,2018,0.428,Africa
33623,Zimbabwe,2019,0.391,Africa
33624,Zimbabwe,2020,0.399,Africa


In [15]:
df_gender_wage_gap = pd.read_csv(r'migracion_charly\gender_wage_gap.csv', sep=',', header=0)

In [16]:
df_gender_wage_gap.drop('INDICATOR', axis=1, inplace=True)
df_gender_wage_gap.drop('MEASURE', axis=1, inplace=True)
df_gender_wage_gap.drop('FREQUENCY', axis=1, inplace=True)
df_gender_wage_gap.drop('Flag Codes', axis=1, inplace=True)

In [17]:
df_gender_wage_gap.rename(columns={'LOCATION':'codigo_pais', 'SUBJECT':'tipo', 'TIME':'anio', 'Value':'valor'}, inplace=True)

In [18]:
df_gender_wage_gap = df_gender_wage_gap[(df_gender_wage_gap['anio'] >= 1990)]

In [19]:
df_gender_wage_gap_employee = df_gender_wage_gap[df_gender_wage_gap['tipo'] == 'EMPLOYEE'].rename(columns={'valor':'wage_gap_employee'}).drop(['tipo'], axis=1)

In [20]:
df_gender_wage_gap_selfemployed = df_gender_wage_gap[df_gender_wage_gap['tipo'] == 'SELFEMPLOYED'].rename(columns={'valor':'wage_gap_selfemployed'}).drop(['tipo'], axis=1)

In [21]:
df_gender_wage_gap_merged = pd.merge(df_gender_wage_gap_employee, df_gender_wage_gap_selfemployed, on=['codigo_pais','anio'], how='outer')

In [22]:
df_gender_wage_gap_merged.head()

,codigo_pais,anio,Wage_gap_employee,Wage_gap_selfemployed
0,AUS,1990,18.181818,NaN
1,AUS,1991,16.015625,NaN
2,AUS,1992,14.258555,NaN
3,AUS,1993,13.224638,NaN
4,AUS,1994,14.409722,NaN


In [24]:
df_gini = pd.read_csv(r'Etl\gini.csv')

In [26]:
df_gini.rename(columns={'Code':'codigo_pais', 'Entity':'pais', 'Year':'anio'}, inplace=True)

In [27]:
merge1 = pd.merge(df_derechos_laborales, df_human_rights, on=['pais','anio'], how='outer')

In [29]:
merge2 = pd.merge(merge1, df_gini, on=['codigo_pais','pais','anio'], how='outer')

In [30]:
fact_ind_sociopolitico = pd.merge(merge2, df_gender_wage_gap_merged, on=['codigo_pais',	'anio'], how='outer')

In [31]:
fact_ind_sociopolitico

,codigo_pais,pais,anio,derechos_laborales,derechos_humanos,region,Gini coefficient,Wage_gap_employee,Wage_gap_selfemployed
0,AGO,Angola,2020,2.27,0.608,Africa,NaN,NaN,NaN
1,AGO,Angola,2019,2.27,0.606,Africa,NaN,NaN,NaN
2,AGO,Angola,2018,2.27,0.602,Africa,0.512721,NaN,NaN
3,ALB,Albania,2020,0.72,0.855,Europe,NaN,NaN,NaN
4,ALB,Albania,2019,0.72,0.872,Europe,0.307717,NaN,NaN
...,...,...,...,...,...,...,...,...,...
9027,EU27,NaN,2018,NaN,NaN,NaN,NaN,10.802786,NaN
9028,EU27,NaN,2019,NaN,NaN,NaN,NaN,10.921490,NaN
9029,EU27,NaN,2020,NaN,NaN,NaN,NaN,10.307368,NaN
9030,ARG,NaN,2021,NaN,NaN,NaN,NaN,6.250000,NaN
